This notebook implements a simple flow described in the following:

cotton - consume - Sew Gown - produce - gown

gown - transfer-custody - gown

gown - accept - Use Gown - modify - gown
              /
         work

gown - transfer-custody - gown

water - consume\
  gown - accept - Clean Gown - modify
soap - consume /                                              

In [ ]:
# Import the necessary packages and also the 'isolation_gowns' notebook to reuse the functions
import json
import requests
import os
import urllib.parse
from zenroom import zenroom
from IPython.core.debugger import set_trace
import re
import base64
from datetime import datetime, timezone
import random
import plotly.graph_objects as go

import import_ipynb
from isolation_gowns import get_filename, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource, get_process, create_event, trace_query, check_trace, er_before, vis_dpp, make_sankey

In [ ]:
# We define the constant for our case
USE_CASE = 'shirts'

# What endpoint are we talking to?
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
ENDPOINT = 'http://65.109.11.42:10000/api'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'


In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []


file = get_filename('cred_users.json',ep=ENDPOINT, uc=USE_CASE)
if os.path.isfile(file):
    with open(file,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['alice'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "Alice",
      "username": "alice_username",
      "email": "alice@example.org",
      "note": "me.alice.org"
    }
    users_data['carol'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Carol",
        "username": "carol_username",
        "email": "carol@example.org",
        "note" : "me.carol.org"
    }

    users_data['bob'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Bob",
        "username": "bob_username",
        "email": "bob@example.org",
        "note" : "me.bob.org"
    }

    with open(file,'w') as f:
        json.dump(users_data, f)

file = get_filename('loc_users.json',ep=ENDPOINT, uc=USE_CASE)
if os.path.isfile(file):
    with open(file,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['one'] = {
        "name": "OLVG",
        "lat": 52.35871773455108,
        "long": 4.916762398221842,
        "addr": "Oosterpark 9, 1091 AC Amsterdam",
        "note": "location.user1.org"
    }
    locs_data['two'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.user2.org"
    }
    with open(file,'w') as f:
        json.dump(locs_data, f)

file = get_filename('units_data.json',ep=ENDPOINT, uc=USE_CASE)
if os.path.isfile(file):
    with open(file,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)


file = get_filename('res_spec_data.json',ep=ENDPOINT, uc=USE_CASE)
if os.path.isfile(file):
    with open(file,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}



In [ ]:
# Read HMAC or get it from the server
endpt_filename = get_filename('cred_users.json',ep=ENDPOINT, uc=USE_CASE)

read_HMAC(endpt_filename, users_data, 'alice')
read_HMAC(endpt_filename, users_data, 'carol')
read_HMAC(endpt_filename, users_data, 'bob')

In [ ]:
# Read the keypair
endpt_filename = get_filename('cred_users.json',ep=ENDPOINT, uc=USE_CASE)

read_keypair(endpt_filename, users_data, 'alice')
read_keypair(endpt_filename, users_data, 'carol')
read_keypair(endpt_filename, users_data, 'bob')

In [ ]:
# read or get id of the person
endpt_filename = get_filename('cred_users.json',ep=ENDPOINT, uc=USE_CASE)

get_id_person(endpt_filename, users_data, 'alice')
get_id_person(endpt_filename, users_data, 'carol')
get_id_person(endpt_filename, users_data, 'bob')

In [ ]:
# Read of get the location id
endpt_filename = get_filename('loc_users.json',ep=ENDPOINT, uc=USE_CASE)

get_location_id(endpt_filename, users_data['alice'], locs_data, 'alice')
get_location_id(endpt_filename, users_data['carol'], locs_data, 'carol')
get_location_id(endpt_filename, users_data['bob'], locs_data, 'bob')

In [ ]:
# Get the ids of all units
endpt_filename = get_filename('units_data.json',ep=ENDPOINT, uc=USE_CASE)

get_unit_id(endpt_filename, users_data['two'], units_data, 'piece', 'u_piece', 'om2:one')
get_unit_id(endpt_filename, users_data['two'], units_data, 'mass', 'kg', 'om2:kilogram')
get_unit_id(endpt_filename, users_data['two'], units_data, 'volume', 'lt', 'om2:litre')
get_unit_id(endpt_filename, users_data['one'], units_data, 'time', 'h', 'om2:hour')


In [ ]:
# Read all the resource specifications
endpt_filename = get_filename('res_spec_data.json',ep=ENDPOINT, uc=USE_CASE)

name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id)

name = 'water'
note = 'Specification for water to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = units_data['volume']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id)

name = 'cotton'
note = 'Specification for cotton to be used to sew the gowns'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id)

name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id)

name = 'surgical_operation'
note = 'Specification for surgical operations'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['time']['id']
get_resource_spec_id(endpt_filename, users_data['two'], res_spec_data, name, note, classification, default_unit_id)

In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'soap'
amount = 100

get_resource(res_data, res_name, users_data['two'], event_seq)

In [ ]:
res_name = 'water'
amount = 50

get_resource(res_data, res_name, users_data['two'], event_seq)

In [ ]:
res_name = 'cotton'
amount = 20

get_resource(res_data, res_name, users_data['two'], event_seq)

In [ ]:
# Create the process that wraps sewing of the gown (its creation)
process_name = 'Sew gown'
user_data = users_data['one']
note = f"Sew gown process performed by {user_data['name']}"

get_process(process_name, note, user_data)

In [ ]:
# Create the process that wraps using the gown in the hospital and make it dirty
process_name = 'Use gown'
user_data = users_data['one']
note = f"Use gown process performed by {user_data['name']}"

get_process(process_name, note, user_data)

In [ ]:
# Create the process that includes cleaning the gown
process_name = 'Clean gown'
user_data = users_data['two']
note = f"Clean gown process performed by {user_data['name']}"

get_process(process_name, note, user_data)

In [ ]:
# Define event consume for the gown creation
action = 'consume'
event_note='consume cotton for sewing'
amount = 10
cur_pros = process_data['Sew_gown']
cur_res = res_data['cotton_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define event produce for the gown creation
action = 'produce'
event_note='produce gown'
amount = 1
cur_pros = process_data['Sew_gown']

res_data['gown_res'] = {
    "res_ref_id": f'gown-{random.randint(0, 10000)}',
    "name": 'gown',
    "spec_id": res_spec_data['gown']['id']
}
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 new_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown from the owner/leaser to the hospital
note='Transfer gowns to hospital'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['two'], action, note, users_data['one'], amount, cur_res)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Work with the gown to perform surgery
action = 'work'
event_note='work perform surgery'
amount = 80
cur_pros = process_data['Use_gown']
work_spec = {}
work_spec['unit_id'] = res_spec_data['surgical_operation']['defaultUnit']
work_spec['spec_id'] = res_spec_data['surgical_operation']['id']

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 work_spec=work_spec)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Use the gown to perform surgery and as a consequence make it dirty
action = 'accept'
event_note='accept use for surgery'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Modify the gown and make it dirty as a consequence of being used
action = 'modify'
event_note='modify dirty after use'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown to the leaser for cleaning
note='Transfer gowns to cleaner'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['one'], action, note, users_data['two'], amount, cur_res)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
action = 'accept'
event_note='accept gowns to be cleaned'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume water for washing
action = 'consume'
event_note='consume water for the washing'
amount = 25
cur_pros = process_data['Clean_gown']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
action = 'consume'
event_note='consume soap for the washing'
amount = 50
cur_pros = process_data['Clean_gown']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# modify the gown that is now clean
action = 'modify'
event_note='modify clean after washing'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 existing_res=cur_res)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
if action in IN_PR_ACTIONS:
    event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})



In [ ]:
show_data()

In [ ]:
# example of using the above functions
# show_resource(users_data['one'], '061Z9FX8M4BJ3M6JMB7MW7VSER')
# show_proposal(users_data['one'], '061Z9FX9H8PT0JF16KP25VS8R0')


In [ ]:
trace = trace_query(res_data['gown_res']['id'], users_data['one'])

In [ ]:
check_trace(trace, event_seq)

In [ ]:
print(f"Resource to be traced: {res_data['gown_res']['id']}")
tot_dpp = []
visited = []
er_before(res_data['gown_res']['id'], users_data['one'], dpp_children=tot_dpp, depth=0)
print(json.dumps(tot_dpp, indent=2))
print(visited)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
vis_dpp(tot_dpp[0],0)
make_sankey(sources, targets, labels, values, color_nodes, color_links)
# make_sankey([0,0,1,2,2], [2,3,3,3,4], ['0','1','2','3','4'], [2,1,1,1,1], color_nodes, color_links)

In [ ]:
values